# Comparison of DFT solvers

We compare four different approaches for solving the DFT minimisation problem,
namely a density-based SCF, a potential-based SCF, direct minimisation and Newton.

First we setup our problem

In [1]:
using DFTK
using LinearAlgebra

a = 10.26  # Silicon lattice constant in Bohr
lattice = a / 2 * [[0 1 1.];
                   [1 0 1.];
                   [1 1 0.]]
Si = ElementPsp(:Si, psp=load_psp("hgh/lda/Si-q4"))
atoms     = [Si, Si]
positions = [ones(3)/8, -ones(3)/8]

model = model_LDA(lattice, atoms, positions)
basis = PlaneWaveBasis(model; Ecut=5, kgrid=[3, 3, 3])

# Convergence we desire in the density
tol = 1e-6

1.0e-6

## Density-based self-consistent field

In [2]:
scfres_scf = self_consistent_field(basis; tol);

n     Energy            log10(ΔE)   log10(Δρ)   Diag   Δtime
---   ---------------   ---------   ---------   ----   ------
  1   -7.846703790379                   -0.70    4.8         
  2   -7.852307383898       -2.25       -1.53    1.0   22.8ms
  3   -7.852612526772       -3.52       -2.56    1.5   26.1ms
  4   -7.852645904623       -4.48       -2.88    2.2   29.3ms
  5   -7.852646478146       -6.24       -3.15    1.0   23.4ms
  6   -7.852646679777       -6.70       -4.13    1.2   23.7ms
  7   -7.852646686574       -8.17       -4.96    2.0   28.9ms
  8   -7.852646686720       -9.84       -5.31    1.8   27.8ms
  9   -7.852646686729      -11.02       -5.90    1.0   85.5ms
 10   -7.852646686730      -12.32       -6.27    1.2   26.0ms


## Potential-based SCF

In [3]:
scfres_scfv = DFTK.scf_potential_mixing(basis; tol);

n     Energy            log10(ΔE)   log10(Δρ)   α      Diag   Δtime
---   ---------------   ---------   ---------   ----   ----   ------
  1   -7.846782654419                   -0.70           4.5         
  2   -7.852524615067       -2.24       -1.63   0.80    2.0    250ms
  3   -7.852636170152       -3.95       -2.70   0.80    1.0   21.6ms
  4   -7.852646264045       -5.00       -3.31   0.80    2.2   27.5ms
  5   -7.852646681849       -6.38       -4.12   0.80    1.8   25.7ms
  6   -7.852646686386       -8.34       -4.76   0.80    1.2   22.2ms
  7   -7.852646686720       -9.48       -5.80   0.80    1.8   25.8ms
  8   -7.852646686730      -11.01       -6.35   0.80    1.8   26.0ms


## Direct minimization
Note: Unlike the other algorithms, tolerance for this one is in the energy,
thus we square the density tolerance value to be roughly equivalent.

In [4]:
scfres_dm = direct_minimization(basis; tol=tol^2);

Iter     Function value   Gradient norm 
     0     1.400173e+01     3.590417e+00
 * time: 0.41134095191955566
     1     1.171274e+00     1.763443e+00
 * time: 0.6031489372253418
     2    -1.421264e+00     2.010111e+00
 * time: 0.6258900165557861
     3    -3.672746e+00     1.675896e+00
 * time: 0.657958984375
     4    -4.965341e+00     1.434118e+00
 * time: 0.689906120300293
     5    -6.792384e+00     1.181156e+00
 * time: 0.7218360900878906
     6    -7.454613e+00     6.938047e-01
 * time: 0.7539451122283936
     7    -7.696137e+00     5.439011e-01
 * time: 0.776515007019043
     8    -7.797735e+00     9.156979e-02
 * time: 0.7990190982818604
     9    -7.825735e+00     1.785446e-01
 * time: 0.8214120864868164
    10    -7.840375e+00     4.938362e-02
 * time: 0.8437061309814453
    11    -7.847895e+00     2.530615e-02
 * time: 0.8659601211547852
    12    -7.851461e+00     1.699257e-02
 * time: 0.8882341384887695
    13    -7.852280e+00     7.664682e-03
 * time: 0.910797119140625

## Newton algorithm

Start not too far from the solution to ensure convergence:
We run first a very crude SCF to get close and then switch to Newton.

In [5]:
scfres_start = self_consistent_field(basis; tol=0.5);

n     Energy            log10(ΔE)   log10(Δρ)   Diag   Δtime
---   ---------------   ---------   ---------   ----   ------
  1   -7.846831147749                   -0.70    4.5         


Remove the virtual orbitals (which Newton cannot treat yet)

In [6]:
ψ = DFTK.select_occupied_orbitals(basis, scfres_start.ψ, scfres_start.occupation).ψ
scfres_newton = newton(basis, ψ; tol);

n     Energy            log10(ΔE)   log10(Δρ)   Δtime
---   ---------------   ---------   ---------   ------
  1   -7.852645894380                   -1.64         
  2   -7.852646686730       -6.10       -3.71    1.53s
  3   -7.852646686730      -13.27       -7.24    157ms


## Comparison of results

In [7]:
println("|ρ_newton - ρ_scf|  = ", norm(scfres_newton.ρ - scfres_scf.ρ))
println("|ρ_newton - ρ_scfv| = ", norm(scfres_newton.ρ - scfres_scfv.ρ))
println("|ρ_newton - ρ_dm|   = ", norm(scfres_newton.ρ - scfres_dm.ρ))

|ρ_newton - ρ_scf|  = 4.549172874404912e-7
|ρ_newton - ρ_scfv| = 1.2440369318549612e-7
|ρ_newton - ρ_dm|   = 5.812157514816788e-10
